# PyTorch를 활용한 객체 인식

## 객체 인식 이론

**객체 인식(Object Detection)**은 이미지 내의 **객체(object)**를 찾아 그 종류와 위치(bounding box)를 동시에 예측하는 과정을 말합니다.

이는 이미지 분류보다 난이도가 높으며, 주로 **bounding box**로 물체의 위치를 표시합니다.

대표적으로 발전해온 객체 검출 기법 몇 가지를 살펴보겠습니다.

### 슬라이딩 윈도우(sliding window)

초기 객체 검출 방법은 이미지 분류기를 기반으로 이미지를 작은 영역으로 잘라가며 모든 위치를 검사하는 방식이었습니다.

예를 들어, 다양한 크기의 윈도우를 이미지를 가로질러 이동(slide)시키면서 각 윈도우 영역에 대해 객체 여부를 분류합니다.

이런 방식은 구현이 비교적 간단하지만, 윈도우 위치/크기 조합이 매우 많아서 계산량이 막대하고 대부분 영역은 객체가 아니므로 비효율적입니다.

### 2 단계 객체 인식 (2-stage object detector)

슬라이딩 윈도우의 비효율을 개선하기 위해 객체일 가능성이 높은 영역만 제안하여 검사하는 기법이 나왔습니다.

예를 들어 Selective Search 같은 알고리즘은 이미지의 시각적 특징을 기반으로 몇 천 개 수준의 후보 영역을 제안합니다.

R-CNN 계열의 방법들은 이러한 제안영역을 사용합니다.

**R-CNN(2014)**은 제안된 각 영역을 CNN 기반 분류기로 분류하고 회귀하여 박스를 조정했습니다.

이후 **Fast R-CNN(2015)**은 한 장의 이미지에서 CNN으로 전체 특징맵을 추출한 뒤, 영역별로 RoI 풀링하여 분류/회귀를 수행하여 속도를 높였고, **Faster R-CNN(2015)**은 아예 CNN 내부에 **Region Proposal Network(RPN)**를 포함시켜 제안영역 추출까지 통합하면서 속도를 크게 개선했습니다.

이러한 2단계(two-stage) 객체 검출기들은 정확도가 높지만 구조가 비교적 복잡하고 실시간 처리에는 느릴 수 있습니다.

### 단일 단계 객체 인식 (1-stage object detector)

더욱 빠른 객체 검출을 위해 단계 구분 없이 바로 한 번의 신경망 전파로 객체 위치와 클래스를 예측하는 기법이 등장했습니다.

대표적으로 **YOLO(You Only Look Once, 2016)**와 **SSD(Single Shot MultiBox Detector, 2016)**가 있습니다.

YOLO는 이미지를 NxN 그리드로 나누고 각 그리드 셀마다 객체의 bounding box와 클래스를 직접 예측합니다.

이 접근법에서는 회귀(regression) 문제로 객체의 좌표를 예측하며, 영역 제안 단계를 없앴기 때문에 실시간 속도로 동작할 수 있습니다.

다만 초기 YOLO는 두 단계 방법에 비해 정확도가 떨어졌으나, 후속 버전(YOLOv2, YOLOv3 등)과 SSD 등의 개선으로 정확도와 속도 모두 향상되었습니다.

Anchor box 등의 개념도 도입되어 다양한 크기의 객체를 한 번에 탐지할 수 있게 했습니다.

요약하면, 슬라이딩 윈도우 -> 영역 제안(R-CNN 계열) -> 단일 단계(YOLO/SSD 계열) 순으로 객체 검출 기술이 발전해왔습니다. 최근에는 Transformer를 이용한 DETR 등 새로운 접근도 등장했지만, 기본 아이디어는 위의 맥락에서 이해할 수 있습니다.

## PyTorch를 활용한 객체 인식 실습

이 강의의 실습에서는 이러한 복잡한 최신 기법들을 직접 구현하지는 않지만, 그 동작 원리를 높은 수준에서 이해하고 간단히 체험해보는 것을 목표로 합니다. 간소화된 객체 검출 실습으로, 하나의 객체에 대한 bounding box를 예측하는 작은 CNN 모델을 만들어보겠습니다.

### 간단한 CNN 회귀 모델로 Bounding Box 예측하기

실습 목표: 이미지 내에 단 하나의 객체가 있을 때, 그 경계상자 좌표를 예측하는 회귀(regression) 모델을 구축합니다. 실제 복잡한 데이터 대신, 이해를 돕기 위해 **합성 데이터(synthetic dataset)**를 생성하여 사용합니다. (예: 검은 배경에 임의의 도형이 그려진 이미지와 그 도형의 bounding box 좌표)

#### 1) 데이터셋 준비 (합성 이미지 및 Bounding Box)

우리는 객체 검출을 위한 간단한 데이터를 직접 생성할 것입니다.

배경이 단색(검정)인 128×128 크기의 이미지에 임의의 도형 하나를 그려 넣고, 그 도형이 "객체"라고 가정합니다.

도형은 원(circle) 또는 사각형(rectangle)으로 하고, 크기와 위치, 색상을 무작위로 변화시켜 다양한 사례를 만듭니다.

각 이미지에는 도형이 하나만 있으며, 해당 도형의 경계상자 좌표 (x_min, y_min, x_max, y_max)를 타겟 값으로 갖습니다.

이렇게 하면 자동으로 입력 이미지와 레이블(bounding box 좌표)을 생성할 수 있습니다.

먼저, 합성 데이터를 생성하는 도우미 함수를 만들어 보겠습니다. Python의 PIL 라이브러리를 사용하여 이미지를 그리고, bounding box 좌표를 반환합니다.

In [ ]:
from PIL import Image, ImageDraw
import numpy as np
import random

def generate_random_shape(img_size=128):
    """img_size x img_size 검정 배경 이미지에 임의의 색상 도형(원 또는 사각형)을 그려넣고
       이미지와 해당 도형의 bounding box 좌표를 반환"""
    # 새 검정색 배경 이미지
    img = Image.new('RGB', (img_size, img_size), color=(0, 0, 0))
    draw = ImageDraw.Draw(img)
    shape_type = random.choice(['rectangle', 'circle'])
    # 임의의 색상 (밝은 색 위주로 선택)
    color_choices = [(255,0,0),(0,255,0),(0,0,255),(255,255,0),(255,0,255),(0,255,255),(192,192,192)]
    color = random.choice(color_choices)
    if shape_type == 'rectangle':
        # 임의 크기와 위치의 사각형
        w = random.randint(img_size//8, img_size//2)
        h = random.randint(img_size//8, img_size//2)
        x1 = random.randint(0, img_size - w)
        y1 = random.randint(0, img_size - h)
        x2 = x1 + w
        y2 = y1 + h
        draw.rectangle([x1, y1, x2, y2], fill=color)
    else:  # circle
        # 임의 크기와 위치의 원 (bounding box로 표시)
        r = random.randint(img_size//10, img_size//4)
        cx = random.randint(r, img_size - r)
        cy = random.randint(r, img_size - r)
        x1 = cx - r
        y1 = cy - r
        x2 = cx + r
        y2 = cy + r
        draw.ellipse([x1, y1, x2, y2], fill=color)
    # 이미지 배열 및 bounding box 좌표 반환
    img_array = np.array(img)  # (H,W,3) numpy 배열
    bbox = np.array([x1, y1, x2, y2], dtype=np.float32)
    return img_array, bbox

# 예시: 하나의 합성 이미지 생성하여 확인
sample_img, sample_bbox = generate_random_shape()
print("생성된 객체 경계상자 좌표:", sample_bbox)

이 함수는 랜덤한 도형과 그 바운딩 박스를 생성합니다.

이제 다수의 이런 이미지를 만들어 학습용/테스트용으로 사용하겠습니다.

학습 데이터 수는 적당히 1000개, 테스트 데이터는 200개 정도로 설정합니다.

In [ ]:
# 합성 데이터셋 생성
num_train = 1000
num_test = 200
train_data = []
train_targets = []
for _ in range(num_train):
    img, bbox = generate_random_shape()
    train_data.append(img)
    train_targets.append(bbox)
test_data = []
test_targets = []
for _ in range(num_test):
    img, bbox = generate_random_shape()
    test_data.append(img)
    test_targets.append(bbox)

train_data = np.array(train_data)        # shape: (1000, 128, 128, 3)
train_targets = np.array(train_targets)  # shape: (1000, 4)
test_data = np.array(test_data)
test_targets = np.array(test_targets)

이제 train_data, train_targets와 test_data, test_targets에 합성된 이미지들과 대응하는 좌표 레이블이 들어갔습니다.

다음으로 이를 PyTorch Dataset으로 만들어 DataLoader로 사용할 수 있게 하겠습니다.

In [ ]:
import torch
class ShapeDataset(torch.utils.data.Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        # 이미지 픽셀을 [0,1]로 정규화하기 위해 255로 나누고 tensor로 변환
        # (H,W,3) ndarray를 (3,H,W) tensor로 변환
        self.data = self.data.astype(np.float32) / 255.0
        self.data = np.transpose(self.data, (0, 3, 1, 2))  # (N,H,W,C) -> (N,C,H,W)
        self.data = torch.from_numpy(self.data)
        self.targets = torch.from_numpy(self.targets)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        # 이미지와 bounding box 좌표 반환
        return self.data[idx], self.targets[idx]

# 학습/테스트 Dataset 및 DataLoader
train_dataset = ShapeDataset(train_data, train_targets)
test_dataset = ShapeDataset(test_data, test_targets)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
print("합성 학습데이터 배치 개수:", len(train_loader))


데이터셋이 준비되었습니다. 이제 본격적으로 bounding box를 예측하는 모델을 구성해보겠습니다.


#### 2) 모델 구현 (Bounding Box 회귀 CNN)


객체 검출에서는 bounding box 좌표를 예측해야 하므로 회귀(regression) 문제로 모델을 훈련시킵니다.

우리 모델은 하나의 물체 좌표를 출력하도록 설계되므로, 최종 출력 차원을 4로 하고 각각 [x_min, y_min, x_max, y_max] 값을 예측하도록 할 것입니다.

CNN 분류기와 유사하게 합성곱 층들을 사용할 수 있지만, 마지막에 Bounding Box 좌표 4개를 출력하기 위해 fully-connected layer의 출력 뉴런 수를 4로 합니다.

그리고 손실 함수로 회귀 손실인 MSE(평균제곱오차)를 사용하겠습니다 (보다 나은 방법으로는 L1 손실이나 IoU 기반 손실 등이 있지만 간단히 MSE로 처리).

모델 구조는 이전 분류기와 유사하게 설계하되, 출력만 4로 바꾸고 활성화 함수를 출력층에는 적용하지 않습니다 (그대로 좌표를 실수로 내보냄).

좌표의 범위는 [0, 128] 픽셀 범위인데, 우리는 정규화하지 않은 픽셀 좌표 자체를 예측하도록 하겠습니다.

필요하면 Sigmoid 등을 써서 [0,1] 사이로 제한한 후 픽셀 크기로 스케일링할 수도 있지만, 여기서는 단순 회귀로 둡니다.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class BoxRegressionCNN(nn.Module):
    def __init__(self):
        super(BoxRegressionCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        # 출력 4개 (bounding box 좌표)
        self.fc = nn.Linear(64 * 16 * 16, 4)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = BoxRegressionCNN().to(device)
criterion = nn.MSELoss()  # 회귀용 손실함수 (Mean Squared Error)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


여기서는 128×128 이미지를 거치면서 conv 세 개와 두 번의 풀링으로 최종 특징맵 크기가 16×16(채널 64)까지 줄었습니다.

Flatten 후 fc로 4차원 출력 (bounding box 좌표) 을 줍니다.

#### 3) 모델 훈련



합성 데이터를 사용하여 bounding box 회귀모델을 훈련합니다.

훈련 루프는 분류 때와 거의 동일하나, 손실 계산 시 레이블과 예측값이 모두 실수 텐서라는 점이 다릅니다.

MSELoss는 예측 좌표와 정답 좌표 간의 차이를 제곱합으로 계산하므로, 이 값을 최소화하도록 학습됩니다.

In [ ]:

num_epochs = 10
model.train()
for epoch in range(1, num_epochs+1):
    running_loss = 0.0
    for images, targets in train_loader:
        images, targets = images.to(device), targets.to(device)
        outputs = model(images)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    print(f"[Epoch {epoch}] 평균 훈련 손실: {avg_loss:.4f}")


출력되는 손실 값이 서서히 줄어드는 것을 확인하세요.

수치적으로 해석하자면, MSE 손실은 예측과 정답 좌표 간 오차 제곱의 평균인데, 예를 들어 손실이 100이라면 대략 좌표 오차가 10 픽셀 정도임을 의미합니다 (왜냐하면 4개 좌표 오차 제곱의 평균이 100이면 하나의 좌표 오차 평균은 sqrt(100)≈10 정도일 수 있음).

손실이 작아질수록 예측 박스가 정답에 가까워지고 있습니다.

#### 4) 모델 평가 및 Bounding Box 결과 시각화



학습된 모델을 평가하기 위해 테스트 세트에서 예측을 수행해보고, Intersection over Union (IoU) 등의 지표를 계산해볼 수 있습니다.

여기서는 간단히 각 이미지마다 예측 박스와 정답 박스의 IoU를 계산하여 평균 IoU를 평가 지표로 삼겠습니다.

IoU는 예측 박스와 정답 박스의 교집합 영역 면적을 합집합 영역 면적으로 나눈 값으로, 두 박스가 얼마나 잘 겹치는지 나타내는 0~1 사이의 수치입니다.

In [ ]:
def compute_iou(box1, box2):
    # box = [x1,y1,x2,y2]
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    # 교집합 영역 좌표가 유효한지 확인
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2]-box1[0]) * (box1[3]-box1[1])
    area2 = (box2[2]-box2[0]) * (box2[3]-box2[1])
    union_area = area1 + area2 - inter_area
    if union_area == 0:
        return 0.0
    return inter_area / union_area

model.eval()
ious = []
with torch.no_grad():
    for images, targets in test_loader:
        images, targets = images.to(device), targets.to(device)
        preds = model(images)
        preds = preds.cpu().numpy()
        targets = targets.cpu().numpy()
        # 각 샘플별 IoU 계산
        for i in range(len(preds)):
            iou = compute_iou(preds[i], targets[i])
            ious.append(iou)
avg_iou = np.mean(ious)
print(f"테스트 세트 평균 IoU: {avg_iou:.3f}")


예를 들어 평균 IoU: 0.85 (85%) 정도로 출력될 수 있습니다.

1.0에 가까울수록 모든 예측 박스가 정답과 거의 일치함을 의미합니다.

이제 몇 개의 테스트 이미지에 대해 예측 결과를 시각화하여 확인해보겠습니다.

이미지에 **실제 경계상자(녹색)**와 **예측 경계상자(빨간색)**를 그려 비교합니다.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# 테스트 이미지 몇 개 선택
samples = 3
fig, axes = plt.subplots(1, samples, figsize=(12,4))
model.eval()
with torch.no_grad():
    for i in range(samples):
        img = test_data[i]
        target = test_targets[i]
        # 모델 예측
        inp = torch.from_numpy(img.astype(np.float32)/255.0).permute(2,0,1).unsqueeze(0).to(device)
        pred = model(inp).cpu().numpy().ravel()
        # 시각화를 위해 이미지 출력
        axes[i].imshow(img)
        # 정답 박스 (녹색) 그리기
        x1, y1, x2, y2 = target
        rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2,
                                 edgecolor='g', facecolor='none')
        axes[i].add_patch(rect)
        # 예측 박스 (빨간색) 그리기
        px1, py1, px2, py2 = pred
        rect2 = patches.Rectangle((px1, py1), px2-px1, py2-py1, linewidth=2,
                                  edgecolor='r', facecolor='none')
        axes[i].add_patch(rect2)
        axes[i].axis('off')
plt.show()


예시로, 한 이미지에서 **실제 경계상자(녹색)**와 **모델이 예측한 경계상자(빨간색)**를 겹쳐 표시한 결과입니다.

대부분 겹쳐 있어 빨간 테두리가 약간 어긋나 보이는 정도로, 모델이 물체의 위치와 크기를 비교적 잘 맞추고 있음을 알 수 있습니다.

이 예시는 하나의 원형 도형에 대한 결과입니다.

위 시각화 결과를 보면, 모델이 학습을 통해 물체의 위치를 어떻게 찾아내는지 감을 잡을 수 있습니다.

우리의 간단한 모델은 항상 하나의 박스만 예측하도록 구성되었지만, 실제 객체 검출 문제에서는 이미지 내 여러 객체를 동시에 찾아야 합니다.

만약 확장하려면 출력으로 여러 박스를 내도록 하거나 (예: 고정 개수의 박스 출력), 신뢰도 점수(confidence score)를 두어 물체 없음인 경우를 거를 수 있어야 합니다.

이러한 복잡성 때문에 실제로는 NMS(Non-Max Suppression) 같은 후처리나 앵커(anchor) 설정 등이 필요하지만, 그 내용은 고급 주제이므로 여기서는 다루지 않습니다.

이번 실습을 통해 회귀를 사용한 객체 위치 예측의 개념을 이해하는 데 집중합니다.